# Annotator Helper
This notebook will use a model to help us annotate images. 

It'll find the images on the vgg file that have no annotations, load those images, apply the model to them, and then will save it back to the json file for further exploration.


In [1]:
import os
from typing import Dict

import torch
import torch.utils.data

from money_counter import data, models, via_utils
from vgg_image_annotation import v2
from PIL import Image, ImageOps
from torchvision import transforms

from typing import Iterable, List
from money_counter.data import CoinsDataset
from money_counter.utils import decode_data


In [2]:
# Load model and weights
model, model_name = models.get_fasterrcnn_pretrained()
version_manager = models.VersionManager('../model_state/')

version_manager.load_model(model_name, model)


Loading model from ../model_state//fasterrcnn_resnet50_fpn-pretrained/epoch_54.pth
Loaded model from ../model_state//fasterrcnn_resnet50_fpn-pretrained/epoch_54.pth


(54, 0.4693695157766342)

In [3]:

# Load dataset
DATASET_DIR = "../assets/coins_dataset"

# Load data from annotations
via_file = v2.load_via_v2_file(f'{DATASET_DIR}/coins.json')

transform = transforms.Compose([
    #transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

def not_annotated(images_metadata: List[v2.ImageMetadata]) -> List[v2.ImageMetadata]:
    return list(filter(lambda image_metadata: not via_utils.is_annotated(image_metadata), images_metadata))


dataset = CoinsDataset('../assets/coins_dataset/coins.json', '../assets/coins_dataset/images/',
                       transform=transform, metadata_transform=not_annotated)


In [4]:
# Create dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, collate_fn=data.collate_into_lists)


In [5]:
# Run the model for the images in the dataset
import json
from typing import Tuple, cast

from torch import Tensor, tensor
from money_counter.models import Target


device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.eval()

def to_shape(tensor: Tensor):
    tensor = tensor.numpy()
    return {
        'name': 'rect',
        'x': float(tensor[0]),
        'y': float(tensor[1]),
        'width': float(tensor[2] - tensor[0]),
        'height': float(tensor[3] - tensor[1])
    }

metadata_partial = []

@torch.no_grad()
def main():
    for images, targets in dataloader:
        # move images to the device for inference
        images = [image.to(device) for image in images]
        # Move targets to the device for inference
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        # Apply model
        outputs = cast(List[Target], model(images))
        # Convert back the outputs to cpu
        outputs = [{k: v.to('cpu') for k, v in cast(
            List[Tuple[str, Tensor]], t.items())} for t in outputs]
            
        # Decode the filename
        images_id = [t['image_id'] for t in targets]
        images_filename = [*decode_data(dataset.filename_map, [i.item() for i in images_id])]
        # Associate to the targets

        outputs = [output | {'filename': filename} 
                for filename, output in zip(images_filename, outputs)]
        
        # Now convert back the bounding boxes to the original format
        outputs = [{
            'filename': output['filename'],
            'regions': [{
                'shape_attributes': to_shape(box),
                'region_attributes': {
                    "Value": "Unknown",
					"Edition": "Unknown",
					"Side": "Unknown",
					"Uncertain": {}
                }
            } for box, score in zip(output['boxes'], output['scores']) if score > 0.35]
        } for output in outputs]

        for output in outputs:
            metadata_partial.append(output)

main()



In [6]:
# Index the metadata
d = {}
for item in metadata_partial:
	d[item['filename']] = item

metadata_partial = d

In [7]:
# load the json with the metadata
via = v2.load_via_v2_file(f'{DATASET_DIR}/coins.json')

for metadata in via['_via_img_metadata'].values():
	if metadata['filename'] in metadata_partial:
		metadata['regions'] = metadata_partial[metadata['filename']]['regions']

# Save the new json
with open(f'{DATASET_DIR}/coins_2.json', 'w') as f:
	json.dump(via, f)

	
